# **Proyecto Final:**
# **Detección de fraudes potenciales en las licitaciones públicas de Chile en el 2020 usando redes bipartitas y el coeficiente de Jaccard**

## Ciencias de Redes
### Profesor: Cristian Candia Vallejos
### Grupo 5:
- Carlos Bustamante
- Pablo Elgueta
- Patricio Ramirez
- Victor Ortiz

---

## Introducción

Una licitación es un proceso administrativo efectuado por un organismo comprador, el cual invita a diferentes proveedores interesados en proporcionar un bien o servicio a realizar una oferta, adeptándose así la más conveniente según los criterios de evaluación descritos en las bases (Mercado público, s.f.). Este proceso es muy vulnerable a la corrupción, debido a que esta puede darse en diferentes etapas y es difícil detectarla. Los principales indicadores de que una licitación podría ser fraudulenta son:

- Reducido número de licitadores
- Ofertas incoherentes de un mismo licitador.
- Similitudes sospechosas entre las ofertas.
- Ofertas no competitivas.
- Patrones de comportamiento similar entre las empresas licitadoras.
- Subcontratación no justificada entre empresas licitadoreas.
- Ofertas presentadas por las mismas personas físicas.
- Ofertas económicas de indéntica redacción, formato o errores.
- Uniones temporales de empresas licitadoras sin justificación aparente.
(Comisión Nacional de los Mercados y la Competencia, 2019)

Para el desarrollo de este trabajo se utilizaron los datos abiertos del portal Chile Compra (https://datos-abiertos.chilecompra.cl/), específicamente los registros del año 2020. Se busca identificar patrones de posibles fraudes utilizando herramientas de análisis de redes. Para conseguir esto, el trabajo se desarrolló en las siguientes etapas:

* Contexto del proyecto.
* Objetivos general, específicos e hipótesis formulada.   
* Análisis exploratorio. 
* Inferencia de la red.
* Descripción de la red.
* Identificación de patrones de corrupción.
* Conclusiones, trabajos futuros y limitaciones.

---

## Contexto del proyecto

Se seleccionaron las licitaciones del año 2020 debido a la sensibilidad que estas tenían respecto a la pandemia, el equipo encargado de desarrollar este proyecto supone que al haber sido una contingencia tan grande, quizás las barreras de control fueron más bajas. En el pasado distintas empresas se han visto envueltas en licitaciones polémicas, por ejemplo LG Electronics y el caso de luminarias públicas en el sur de Chile (CIPER, 2015), los casos PagoGate y MilicoGate (El Mostrador, 2019) y un sinfín de otros organismos tanto públicos como privados.

La base de datos contiene las siguientes columnas:

* Codigo: número único de licitación.                      
* Tipo de Adquisicion: rango del monto de la licitación.
* Estado: si la licitación se declaró adjudicada, desierta, revocada, cerrada o suspendida.
* CodigoOrganismo: número único del organismo público que abre una licitación. 
* NombreOrganismo: nombre del organismo público que abre la licitación.
* RutUnidad: rol único tributario de la unidad que participa en la licitación.
* ComunaUnidad: nombre de la comuna en la que la licitación es requerida.
* FechaPublicacion: fecha en la que la licitación se publicó.
* CodigoProductoONU: sistema de codificación común de las naciones unidas.
* RutProveedor: rol único tributario de la empresa que postula a la licitación.
* NombreProveedor: nombre de la empresa que se postula a la licitación.
* Moneda de la Oferta: divisa en la que se expresa el monto a pagar de la licitación.
* MontoLineaAdjudica: monto que se adjudica la empresa en la licitación, puede ser total o parcial.
* Oferta seleccionada: indicador booleando que expresa si la oferta efectuada por la empresa fue seleccionada o no.

## Objetivos general, específicos e hipótesis formulada

**Objetivo General**

Encontrar patrones de fraude y corrupción en las licitaciones públicas de Chile registradas en el año 2020.

**Objetivos Específicos**

- Desarrollo de una red bipartita que integre organismos y empresas.
- Identificar la componente gigante de la red.
- Identificar patrones de corrupción usando herramientas de análisis de redes complejas.

**Hipótesis Formulada**

¿Existe una relación que se repita de forma sostenida entre Organismos Públicos con los Proveedores para el periodo a analizar (2020)?.  

- La hipótesis del grupo es que la cantidad de interacciones entre un organismo y un proveedor respecto del global es un indicador de fraude.

- El hecho de que pocos proveedores se postulen a un concurso público, podría sugerir que el organismo no está publicando de manera justa sus licitaciones.



## Análisis exploratorio

In [1]:
# Librerías

#Matemática y datos
import pandas as pd
import numpy as np

#Visualización
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

#Redes
import networkx as nx
from networkx.algorithms import bipartite
import community
from community import community_louvain


In [2]:
# Leer datos

df = pd.read_csv('lic2020_hackaton.csv', sep=',', encoding='latin_1')
df['FechaPublicacion']=pd.to_datetime(df['FechaPublicacion'])
df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'lic2020_hackaton.csv'

En primer lugar se debe analizar el contenido y eliminar aquellas filas que contuvieran datos nulos. Esto debido a que al momento de querer enlazar nodos o identificar patrones comunes estos datos quedan vinculados.

In [ ]:
df.isnull().sum()

In [ ]:
df[df['NombreProveedor'].isnull()].head(3)

In [ ]:
df = df.dropna()
df.isnull().sum()

- De acuerdo a la información contenida en el dataframe, es posible identificar que claramente existen 2 grupos de nodos. 

-Los primeros con sus datos agrupados en las columnas de la izquierda, corresponden a los organismos públicos que generan las licitaciones.


- En las columnas de la derecha figura lo referente a las empresas proveedoras que participan de las licitaciones asociadas a los organismos públicos, estableciéndose de este modo los enlaces para esta red bipartita.


- Identificados los grupos de nodos de la red bipartita, se procede a definir las columnas a utilizar de las que se extraerán estos, si utilizar códigos o nombres. Dado que en los Organismos las cantidades de Nombres son inferiores a las de Códigos, se opta por utilizar los Nombres, ya que eventualmente podría existir más de un Código asociado a los organismos, lo cual generaría una distorsión en el análisis al momento de establecer relaciones.

    

In [ ]:
print('Cantidad de Nombres de Organismo:')
print(len(df['NombreOrganismo'].unique()))
print('\nCantidad de Codigos de Organismo:')
print(len(df['CodigoOrganismo'].unique()))

In [ ]:
# Cantidad de Licitaciones abiertas por mes

df11 = df[df.Codigo.duplicated(keep=False)]
df11 = df11['FechaPublicacion'].dt.date.value_counts().sort_index().reset_index()
df11.columns=['Fecha Publicacion','Codigo']
df11['Fecha Publicacion']=pd.to_datetime(df11['Fecha Publicacion'])
df11['month'] = df11['Fecha Publicacion'].dt.month
df12 = df11.groupby(df11['Fecha Publicacion'].dt.month)['Codigo'].sum()
df12 = df12.to_frame().reset_index()
df12.columns = ['Mes', 'Cantidad Licitaciones']
ax = df12.plot.bar(x='Mes', y='Cantidad Licitaciones', rot=0)

Observando la distribución de  la cantidad de licitaciones durante el año, se puede observar una tendencia negativa, la cual no presentó mayores variaciones entre un mes y otro, a excepción de los primeros 3 meses del año. Esta variación no es significativa como para centrar incialmente un análisis en este punto.

In [ ]:
# Cantidad total de veces que algún proveedor aplicó a alguna licitación

df13 = df['NombreProveedor'].value_counts().to_frame().reset_index()
df13['NombreProveedor'].sum()

In [ ]:
# Top proveedores que más aplicaron a licitaciones

df13['Porcentaje del total'] = df13['NombreProveedor']/1769860
df13.rename(columns = {'index':'Proveedor','NombreProveedor':'Conteo'}, inplace = True)
df13[:15]

Tal como se aprecia en la tabla, ninguno de los proveedores concentra un alto porcentaje de participación en licitaciones como para iniciar un análisis respecto a estos valores.

In [ ]:
# Cantidad total de veces que algún organismo abrió una licitación

df14 = df['NombreOrganismo'].value_counts().to_frame().reset_index()
df14['NombreOrganismo'].sum()

In [ ]:
# Top Organismos que más licitaciones abrieron

df14['Porcentaje del total'] = df14['NombreOrganismo']/1769860
df14.rename(columns = {'index':'Organismo','NombreOrganismo':'Conteo'}, inplace = True)
df14[:15]

En general es posible apreciar que no existe una entidad que concentre un porcentaje de licitaciones significativamente mayor a las demás como para ser objeto de análisis. 

In [ ]:
# Conteo de ofertas seleccionadas

df15 = df['Oferta seleccionada'].value_counts().to_frame().reset_index()
df15.rename(columns = {'Oferta seleccionada':'Conteo','index':'Selección ofta'},inplace = True)
df15

In [ ]:
# Top proveedores que más fueron seleccionados

df15 = df[df['Oferta seleccionada'] == 'Seleccionada']
df15 = df15['NombreProveedor'].value_counts().to_frame().reset_index()
df15['Porcentaje del total'] = df15['NombreProveedor']/359922
df15.rename(columns = {'index':'Proveedor','NombreProveedor':'Conteo'}, inplace = True)
df15[:15]

Dentro de los proveedores con mayor cantidad de adjudicaciones se aprecia que predominan aquellos vinculados a la industria farmacéutica, sin embargo, individualmente ninguno arroja un porcentaje lo suficientemente grande apra ser objeto de análisis.

In [ ]:
# Conteo del estado de la oferta

df16 = df['Estado'].value_counts().to_frame().reset_index()
df16.rename(columns = {'Estado':'Conteo','index':'Estado'},inplace = True)
df16

In [ ]:
# Top proveedores que más licitaciones se adjudicaron

df16 = df[df['Estado'] == 'Adjudicada']
df16 = df16['NombreProveedor'].value_counts().to_frame().reset_index()
df16['Porcentaje del total'] = df16['NombreProveedor']/1633154
df16.rename(columns = {'index':'Proveedor','NombreProveedor':'Conteo'}, inplace = True)
df16[:15]

In [ ]:
df_te= df13.merge(df15, how='inner', on='Proveedor')
df_te = df_te[df_te['Conteo_y'] > 1000]
df_te['Tasa éxito'] = df_te['Conteo_y'] / df_te['Conteo_x']
df_te = df_te.reset_index().sort_values(by='Tasa éxito', ascending=False)
df_te.rename(columns = {'Conteo_x':'Conteo Postulaciones','Porcentaje del total_x':'% del total Postulaciones',
                       'Conteo_y':'Conteo Seleccionadas','Porcentaje del total_y':'% del total Seleccionadas'}, inplace = True)
df_te



- Del análisis de datos, fue posible identificar que existen 2 grupos de nodos mediante los cuales es factible generar una Red Bipartita. El primer Grupo contempla las primeras columnas de datos que se encuentras asociadas a los Organismos estatales que generan las Licitaciones. Por otro lado, el conjunto de columnas de la izquierda corresponden a las Empresas participantes de las licitaciones.

- Dentro del análisis exploratiorio de los datos, fue posible identificar que las cantidades de CodigosOrganismos no coincidía con la de NombreOrganismos, lo cual eventualmente significaría que existen Organismos con más de un código asociado, lo que nos inclina por trabajar con los Nombres.

- Otra observación que pudimos identificar, es que algunas de las empresas que participan como oferentes, se 
encuentran también dentro del listado de Organismos, estas corresponden a aquellas empresas estatales como universidades, las cuales poseen áreas de proyectos o investigación para el autofinanciamiento.

- Resulta interesante ver las altas tasas de éxito que tienen algunas empresas respecto de las otras al quedar seleccionados en las licitaciones. El top 7 posee una tasa mayor al 50%, pero con las empresas que vienen después esa tendencia desaparece absolutamente.

## Inferencia de la red
### Creación de la red:

In [ ]:
# Creación de una lista de nodos con los organismos públicos

lista_organismos = df['NombreOrganismo'].unique()
lista_organismos[0:5]
len(lista_organismos)

In [ ]:
# Lista de nodos de empresas

lista_empresas = df['NombreProveedor'].unique()
lista_empresas[0:5]
len(lista_empresas)

Dado que existen empresas que figuran tanto como organismo público y como proveedor, afectando la creación de la red bipartita, se identifican estas instituciones y posteriormentes son eliminadas del dataset. Estas entidades corresponden a empresas estatales como universidades, las cuales poseen áreas de proyectos o investigación para el autofinanciamiento.

In [ ]:
del_empresas = []
for organismo in lista_organismos:
    for empresa in lista_empresas:
        if organismo == empresa:
            del_empresas.append(organismo)
del_empresas

for universidad in del_empresas:
    df = df[df['NombreProveedor'] != universidad]

lista_empresas = df['NombreProveedor'].unique()
len(lista_empresas)

In [ ]:
del_empresas

In [ ]:
# Conteo de repeticiones organismo-proveedor

pivot = pd.pivot_table(df, index=['NombreOrganismo','NombreProveedor'],values=['Oferta seleccionada'],aggfunc='count')
pivot = pivot.reset_index().sort_values(by='Oferta seleccionada', ascending=False)
pivot.rename(columns = {'Oferta seleccionada':'Conteo Interacciones'}, inplace = True)
pivot.head(15)

Para la creación de la red se buscan los pares de nodos con su respectivo peso, en este caso sería la cantidad de interacciones que tiene un Organismo con un Proveedor de manera individual.

Siguiendo la línea de calcular las coincidencias, se realizó un conteo para el dataset filtrado por empresas seleccionadas, lo cual dió pistas claras para investigar, sin embargo, esa parte del proyecto será abordada más adelante.

In [ ]:
# pivot sólo proveedores seleccionados
df15 = df[df['Oferta seleccionada'] == 'Seleccionada']

pivot15 = pd.pivot_table(df15, index=['NombreOrganismo','NombreProveedor'],values=['Oferta seleccionada'],aggfunc='count')
pivot15 = pivot15.reset_index().sort_values(by='Oferta seleccionada', ascending=False)
pivot15.rename(columns = {'Oferta seleccionada':'Conteo Seleccionados'}, inplace = True)
pivot15.head(15)

In [ ]:
# Se crean tuplas con los pares organismo-proveedor y peso, a modo de encale entre los nodos

lista_tuplas_pesos = []

for i in pivot.to_numpy():
    lista_tuplas_pesos.append((i[0], i[1], i[2]))
lista_tuplas_pesos[:4]

In [ ]:
len(lista_tuplas_pesos)

In [ ]:
# Se construye el gráfo de red bipartita

G = nx.Graph()
G.add_nodes_from(lista_empresas, bipartite=0)
G.add_nodes_from(lista_organismos,bipartite=1)
G.add_weighted_edges_from(lista_tuplas_pesos)
bipartite.is_bipartite(G)

### Análisis de la red:

In [ ]:
N1 = len(G)
L1 = G.size()
degrees1 = list(dict(G.degree()).values())
kmin1 = min(degrees1)
kmax1 = max(degrees1)
densidad = nx.density(G)
print("Número de nodos: ", N1)
print("Número de enlaces: ", L1)
print('-------')
print("Grado promedio: ", 2*L1/N1) #Formula vista en clases (qué sucedía con las redes reales?)
print("Grado promedio (alternativa de calculo)", np.mean(degrees1))
print('-------')
print("Grado mínimo: ", kmin1)
print("Grado máximo: ", kmax1)
print('-------')
print("Densidad de la red: ", densidad)

Es posible apreciar que la densidad de la red es bastante baja, lo cual también permite inferir que la red poseería varias comunidades entre las cuales se relacionen los nodos.

In [ ]:
# Distribución de grado de la red

degrees = list(dict(G.degree()).values())
plt.hist(list(degrees))
plt.show()

In [ ]:
# Entrega 20 bins linealmente espaceados entre kmin y kmax
bin_edges = np.linspace(kmin1, kmax1, num=30)

# histograma de la data en estos bines 
density, _ = np.histogram(degrees1, bins=bin_edges, density=True)
###Histograma
fig = plt.figure(figsize=(6,4))

# "x" debería ser el punto medido (en escala lineal) de cada bin
log_be = np.log10(bin_edges)
x = 10**((log_be[1:] + log_be[:-1])/2)

plt.plot(x, density, marker='o', linestyle='none')
plt.xlabel(r"degree $k$", fontsize=16)
plt.ylabel(r"$P(k)$", fontsize=16)

# Muestra la gráfica
plt.show()

In [ ]:
bin_edges = np.logspace(np.log10(kmin1), np.log10(kmax1), num=30)
density, _ = np.histogram(degrees1, bins=bin_edges, density=True)
np.histogram(degrees1, bins=bin_edges, density=True)

###Histograma
fig = plt.figure(figsize=(6,4))

# "x" debe ser el punto medio (en escala LOG) de cada bin
log_be = np.log10(bin_edges)
x = 10**((log_be[1:] + log_be[:-1])/2)

plt.loglog(x, density, marker='o', linestyle='none')
plt.xlabel(r"degree $k$", fontsize=16)
plt.ylabel(r"$P(k)$", fontsize=16)

# Muestra la gráfica
plt.show()

El análisis mediante histogramas permite confirmar nuevamente que existe una gran cantidad de nodos poco conectados y que para grados mayores la distribución de nodos disminuye exponencialmente.

### Identificacion de Comunidades:

- Tal como era posible predecir debido a lo poco densa de la red, existe una cantidad significativa de comunidades.


- En este punto se ha optado por definir un random_state para la función de best_partition que arroje una partición con varias comunidades de baja cantidad de nodos, a fin de facilitar el análisis de las comunidades mediante algoritmos.

In [ ]:
partition = community_louvain.best_partition(G, weight='weight', random_state = 6)

size = (len(set(partition.values())))#Numero de comunidades
print('Se detectan %d comunidades' % (size))
print('\n')

d = {}
for character, par in partition.items():
    if par in d:
        d[par].append(character)
    else:
        d[par] = [character]

for i in range(len(d)):
    print(f'Tamaño Comunidad {i}: {len(d[i])}')

### Nota:

###### Dado que correr el código nx.spring_layout(G) tarda demasiado tiempo, se ha generado un código que permite importar ya la disposición de nodos desde un archivo CSV.  

###### Se recomienda desactivar o eliminar los siguientes códicos si el programa ya ha sido ejecutado previamente y el archivo CSV ha sido generado, a fin de agilizar su ejecución:


- pos = nx.spring_layout(G) # Layout para la red (coordenadas de los nodos y enlaces)
- pos_export = pd.DataFrame(pos)
- pos_export.to_csv('TareaFinalPos.csv')

In [ ]:
pos = nx.spring_layout(G) # Layout para la red (coordenadas de los nodos y enlaces)
pos_export = pd.DataFrame(pos)
#pos_export.to_csv('TareaFinalPos.csv')
#pos_import = pd.read_csv('TareaFinalPos.csv', sep=',')
pos_import = pos_export
#pos_import = pos_import.drop(['Unnamed: 0'], axis=1)
cord_dict = {}
for key in pos_import:
    cord = [pos_import[key][0], pos_import[key][1]]
    cord = np.array(cord)
    cord_dict[key] = cord
    
pos_import = cord_dict

In [ ]:
#Posiciones definidas para los nodos
len(pos_import)

In [ ]:
#Verificacion de cantidad de nodos
len(lista_organismos)+len(lista_empresas)

In [ ]:
#Corroborar que la suma de cada nodo separado en als comunidades coincidan con la cantidad de nodos

sum = int()
for i in range(len(d)):
    sum = sum + len(d[i])
print (sum)

#### Ploteo del Grafo

In [ ]:
plt.figure(figsize=(25, 15))

#colors = [np.array(cm.jet(x)).reshape(1,-1) for x in np.linspace(0, 1, size)]#cm.jet es el mapa de colores https://www.programcreek.com/python/example/56498/matplotlib.cm.jet
colors = ['white', 'white', 'white', 'yellow', 'white', 'white', 'white', 'white', 'white', 'white', 'cyan', 'orange', 'blue', 'white', 'green', 'red', ]

count = 0
for com in set(partition.values()): #para cada comunidad
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos_import, list_nodes, node_size = 20, node_color=colors[count], alpha=0.8)#plotea nodos con colors por comunidad
    count = count + 1# para iterar sobre los colores

nx.draw_networkx_edges(G, pos_import, alpha=0.5, width=0.15)#plotea enlaces
plt.show()


Mediante el gráfico de red es posible notar que si bien existen comunidades pequeñas que se notan aisladas en la periferia, se denota que las comunidades mas grandes no son facilmente separables entre si, por lo cual se deduce que la partición debe tener baja modularidad.

## Identificación de patrones de corrupción

In [ ]:
# Funcion para crear lista de codigos de licitacion entre un par organismo-proveedor

def lista_empresa_licitacion(nombre_organismo, nombre_proveedor):
    
    df_ddn = df[['Codigo','NombreOrganismo','NombreProveedor','Oferta seleccionada']]
    df_ddn = df_ddn[(df_ddn['NombreProveedor'] == nombre_proveedor) ]
    
    lista_codigos = []

    for i in df_ddn['Codigo'].to_numpy():
        lista_codigos.append(i)
       
    return lista_codigos

# Funcion para conseguir el coeficiente de jaccard

def jaccard(grupo1, grupo2):
    interseccion = len(set(grupo1).intersection(set(grupo2)))
    union = len (set(grupo1).union(set(grupo2)))
    
    return interseccion / union , interseccion

# Funcion para conseguir el coeficiente de jaccard para todos los nodos dentro de una comunidad

def jaccard_completo(dd):
    
    lista_nombres_organismos = dd['NombreOrganismo']
    lista_nombres_proveedores = dd['NombreProveedor']
    
    dicc_codigos_dd = {}
    
    df = pd.DataFrame(columns=['NombreProveedor_1', 'NombreProveedor_2', 'Coeficiente de Jaccard', 'CantidadCoincidencias'])
    
    for i in range(len(dd)):
        lista_codigos = lista_empresa_licitacion(lista_nombres_organismos[i], lista_nombres_proveedores[i])
        
        dicc_codigos_dd[lista_nombres_proveedores[i]] = lista_codigos
                
    for key_j, value_j in dicc_codigos_dd.items():
        for key_k, value_k in dicc_codigos_dd.items():
            jacard_empresas, interseccion = jaccard(value_j, value_k)
            #print (f"Porcentaje de similitud Para Empresas {key_j} y {key_k} : {round(jacard_empresas,2)}")
            df_append = pd.DataFrame({'NombreProveedor_1': key_j, 'NombreProveedor_2': key_k,
                                      'Coeficiente de Jaccard': round(jacard_empresas,2), 'CantidadCoincidencias':interseccion }, index=[0])
            frames = [df, df_append]
            df = pd.concat(frames)
            
    df = df.sort_values(by='Coeficiente de Jaccard', ascending=False)
    df = df.reset_index().drop(columns='index')
    
    for index, row in df.iterrows():
        if row['NombreProveedor_1'] == row['NombreProveedor_2']:
            df = df.drop(index)
    df = df.reset_index().drop(columns='index')
    df2 = df
    for i in range(len(df2)):
        for k in range(i+1, len(df2)):
            
            if df2.NombreProveedor_1[i] == df2.NombreProveedor_2[k]: 
                if df2.NombreProveedor_1[k] == df2.NombreProveedor_2[i]:
                    df = df.drop(k)
        
    return df    

In [ ]:
def comunidad(df, d):

    df2 = pd.DataFrame()
    j = 0
    for i in range (len(df)):
        if df.NombreOrganismo[i] in d:
            if j == 0:
                df2 = df.loc[i:i]
                j = 1
            else:
                df3 = df.loc[i:i]
                frames = [df2,df3]
                df2 = pd.concat(frames)
    df2 = df2.reset_index()
    
    j = 0
    for i in range (len(df2)):
        if df2.NombreProveedor[i] in d:
            if j == 0:
                df1 = df2.loc[i:i]
                j = 1
            else:
                df3 = df2.loc[i:i]
                frames = [df1,df3]
                df1 = pd.concat(frames)
    df1 = df1.reset_index()                        
    df1 = df1[['NombreOrganismo','NombreProveedor','Conteo Interacciones']]
    
    lista_tuplas_pesos = []
    for i in df1.to_numpy():
        lista_tuplas_pesos.append((i[0], i[1], i[2]))
    lista_tuplas_pesos[:4]

    G = nx.Graph()
    G.add_nodes_from(df1['NombreProveedor'].unique(), bipartite=0)
    G.add_nodes_from(df1['NombreOrganismo'].unique(),bipartite=1)
    G.add_weighted_edges_from(lista_tuplas_pesos)
    
    plt.figure(figsize=(10, 6))
    
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, d, node_size = 20, node_color=['red'], alpha=0.8)
    nx.draw_networkx_edges(G, pos, alpha=0.8, width=0.25)

    plt.show()
            
    return df1, G

### Comunidad de estudio #1: Gobierno Regional de Atacama

Para esta comunidad se realizaron dos análisis: el primero mediante la representación gráfica de la red bipartita y la tabla de enlaces ponderados, para poder definir si existe algún tipo de vinculación entre el organismo licitador y las empresas proveedoras. El segundo es el cálculo del coeficiente de Jaccard, para identificar la razón de coincidencias entre las licitaciones en las que participan los nodos de la comunidad.

In [ ]:
dd0,Gd0 = comunidad(pivot, d[13])
dd0.sort_values(by='Conteo Interacciones', ascending=False)

Es posible apreciar que existe una llamativa distribución en las particiones dentro de esta comunidad, donde un total de 25 participaciones en licitaciones corresponden a solo 2 códigos con un total de 12 OC emitidas.

In [ ]:
df0 = df[df['NombreOrganismo'] == 'GOBIERNO REGIONAL DE ATACAMA']
df0 = df0[['Codigo','Tipo de Adquisicion','NombreOrganismo','NombreProveedor','MontoLineaAdjudica','Moneda de la Oferta']]
df0

In [ ]:
df0[df0['Codigo']== 8748588]['MontoLineaAdjudica'].astype('int64', copy=False).sum()

- Dentro de las 12 OC emitidas por el Gobierno Regional de Atacama, 11 son a "GYP TECNOLOGIA" con un valor total de $355.400, donde el proceso de licitación contó con solamente un oferente adicional.

- Respecto a la OC restante , en su proceso de licitación sólo participaron 3 oferentes y el proveedor adjudicado fue "Zerega y Cia.Ltda" por un monto de $19.800.000.

In [ ]:
codigos = jaccard_completo(dd0)
codigos[codigos['CantidadCoincidencias']>0.9*codigos['CantidadCoincidencias'].max()]

- Este indicador permite identificar que en el caso de "Aquapress Limitada" y "Zerega Y Cia. Ltda." dentro de los registros solo poseen una única participación en licitaciones y corresponde a la del Gobierno Regional de Atacama.


- Si bien ambos procesos de adjudicación parecen llamativos, en el caso de la licitación 8682225 adjudicada a "Zerega Y Cia. Ltda." es mucho más llamativo ya que por un lado la licitación posee muy pocos oferentes y 2 de ellos presentan solo una única participación lo cual podría ser indicio de colusión o de participantes que solo buscaban completar la terna para que la licitación no sea declarada desierta.

### Comunidad de estudio #2: Hospital Padre Alberto Hurtado

In [ ]:
dd1,Gd1 = comunidad(pivot, d[14])
dd1.sort_values(by='Conteo Interacciones', ascending=False)

In [ ]:
codigos1 = jaccard_completo(dd1)
codigos1[codigos1['CantidadCoincidencias']>0.9*codigos1['CantidadCoincidencias'].max()]

In [ ]:
df1 = df[df['NombreOrganismo'] == 'HOSPITAL PADRE ALBERTO HURTADO'][df['Oferta seleccionada']=='Seleccionada']
df1 = df1[['Codigo','Tipo de Adquisicion','NombreOrganismo','NombreProveedor','MontoLineaAdjudica','Moneda de la Oferta']]
df1.head(5)

In [ ]:
len(df1)

### Comunidad de estudio #3: Municipalidad Freirina, Municipalidad Hualaihue y SS Hospital Quirihue

In [ ]:
dd2,Gd2 = comunidad(pivot, d[4])
dd2.sort_values(by='Conteo Interacciones', ascending=False)

In [ ]:
codigos2 = jaccard_completo(dd2)
codigos2[codigos2['CantidadCoincidencias']>0.9*codigos2['CantidadCoincidencias'].max()]

- Esta comunidad tiene un aspecto sano, si bien la tabla anterior nos muestra que existen proveedores con un alto coeficiente de Jaccard, si se observa la cantidad de licitaciones asociadas a cada uno de estos 3 organimos es posible apreciar que la cantidad de licitaciones en que los proveedores "BRAULIO RODRIGO", "María José" o "ALVEAL121" coinciden son muy pocas en relacion al total de licitaciones.


- En general, el hecho de que nodos correspondientes a Proveedores conecten nodos correspondientes a Organismos, proporciona un sintoma de sanidad en la comunidad a diferencia de lo ocurrido en el caso del Gobierno Regional de Atacama.

### Comunidad de estudio #4: Caso MOSIL LIMITADA

In [ ]:
df101 = df[df['NombreProveedor'] == 'MOSIL LIMITADA'][df['Oferta seleccionada']=='Seleccionada']
df101 = pd.pivot_table(df101, index=['NombreOrganismo','NombreProveedor'],values=['Oferta seleccionada'],aggfunc='count')
df101 = df101.reset_index().sort_values(by='Oferta seleccionada', ascending=False)
df101.rename(columns = {'Oferta seleccionada':'Conteo Seleccionados'}, inplace = True)
df101.head(15)

In [ ]:
df102 = df[df['NombreOrganismo'] == 'I MUNICIPALIDAD DE LOTA']
lota = []

for i in df102['NombreProveedor'].unique():
    lota.append(i)
    
lota.append('I MUNICIPALIDAD DE LOTA')

In [ ]:
lota

In [ ]:
ddlota,G_lota = comunidad(pivot, lota)
ddlota = ddlota.sort_values(by='Conteo Interacciones', ascending=False)

In [ ]:
ddlota.reset_index().drop(columns='index')

In [ ]:
dflota = df[df['NombreProveedor'] == 'MOSIL LIMITADA'][df['Oferta seleccionada']=='Seleccionada']
dflota

In [ ]:
ddlota_aux = ddlota.iloc[:10]
ddlota_aux = ddlota_aux.reset_index().drop(columns='index')
codigos123 = jaccard_completo(ddlota_aux)
codigos123[codigos123['CantidadCoincidencias']>0.001*codigos123['CantidadCoincidencias'].max()]

La empresa MOSIL LIMITADA no tiene un coeficiente de Jaccard alto, por lo que dentro de la Municipalidad de Lota está bien distribuida, sin embargo, es llamativo que la tasa de éxito en esta empresa sea tan alto.

## Conclusiones, trabajos futuros y limitaciones

### Conclusiones

- Las empresas farmaceuticas son el giro el cual concentra la mayor cantidad de licitaciones en el año 2020.


- Se esperaba que hubieran mas instituciones de salud en los organismos que abrieron licitaciones, pero este top lo lideran instituciones de fuerzas armadas.


- Se analizó la tasa de éxito de los proveedores, encontrando una distribución sospechosa entre el top 7 y que la empresa MOSIL es potencialmente fraudulenta.


- Se construyó una red bipartita con 38.617 nodos y 205.391 enlaces.


- Observando la distribución de grado, se ve que gran cantidad de nodos bajamente conectados y que para grados mayores la distribución de nodos disminuye exponencialmente.


- El cálculo de comunidades mediante el algoritmo de Louvain es una excelente herramienta para detectar posibles casos de corrupción. Se estudiaron las comunidades más pequeñas encontrando anomalías importantes en cuanto a los criterios de selección de proveedores por parte los organismos.


- El coeficiente de Jaccard es también un buen indicador el cual muestra cuantas veces coincidieron los mismos proveedores en las licitaciones. Si el coeficiente es bajo, significa que un proveedor está postulando de manera aislada a las licitaciones sin competir.


- En el caso de estudio de Atacama, todos los participantes tienen un alto coef Jaccard entre sí. Los ganadores tienen un coef Jaccard bajo sugiriendo que estas empresas no competían.


- En el caso de MOSIL LIMITADA, quien destaca por su gran cantidad de licitaciones aceptadas (comparables con empresas farmacéuticas de renombre) y una de las tasas de éxito postulación/aceptación mas altas de todos los proveedores, concentra en la Municipalidad de Lota 2.000 licitaciones aceptadas, siendo esto una anomalía importante comparándolo con el general del dataset. Al calcular su Coeficiente de Jaccard, el grupo se percató de que coincide en licitaciones con la empresa Salcobrand, siendo MOSIL una empresa de construcción.


- La comunidad del Hospital Padre Hurtado era pequeña pero al analizarse no identificamos patrones de fraude.


- Se concluye que el desafío fue correctamente superado al haber finalizado cada una de las etapas solicitadas con éxito.


- Resulta muy positivo que la actividad final de esta asignatura contemple el desarrollo de un proyecto real, poniendo en práctica las herramientas adquiridas en esta esta asignatura y también en el magister.


### Trabajos futuros

- Existe una oportunidad al automatizar las tareas de análisis manual efectuadas durante este desafío.


- Una actividad para profundizar esta investigación, sería repetir el análisis desarrollado tomando en cuenta las comunidades mas grandes y descomponiendolas en sub-comunidades mas pequeñas, para así poder ir identificando patrones de corrupción de manera cíclica.


### Limitaciones

- La capacidad computacional enletece los análisis y lo hace en algunos casos imposible debido a la gran cantidad de ram necesaria para procesar esta red


- La cantidad de tiempo óptima para realziar un análisis realmente profundo de la base de datos es mayor a la que el grupo tuvo para finalizar el desafío.


### Bibliografía y Webgrafía

- https://www.mercadopublico.cl/Home/Contenidos/QueEsLicitacion
- https://www.cnmc.es/guias
- https://www.ciperchile.cl/2015/08/11/el-fraude-que-oscurece-las-licitaciones-que-gano-lg-para-iluminar-seis-comunas-con-led/
- https://www.elmostrador.cl/noticias/pais/2023/01/14/a-7-anos-del-milicogate-las-agencias-de-viajes-involucradas-se-adjudican-millonarias-licitaciones-con-el-estado/